In [ ]:
%install-location $cwd/swift-install
%install '.package(path: "$cwd/FastaiNotebook_10_mixup_ls")' FastaiNotebook_10_mixup_ls
// %install '.package(path: "$cwd/FastaiNotebook_00_load_data")' FastaiNotebook_00_load_data
// %install '.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")' Path
// %install '.package(url: "https://github.com/JustHTTP/Just", from: "0.7.1")' Just
// %install '.package(url: "https://github.com/mechamoth/SwiftFFmpeg", from: "1.5.0")' SwiftFFmpeg 
// %install '.package(url: "https://github.com/mechamoth/WAVCat", .branch("cf2eca3"))' WAVCat

Installing packages:
	.package(path: "/home/jupyter/git/fastai_docs/dev_swift/FastaiNotebook_10_mixup_ls")
		FastaiNotebook_10_mixup_ls
With SwiftPM flags: []
Working in: /tmp/tmpiqhmjw2u/swift-install
/home/jupyter/swift/usr/bin/swift-build: /home/jupyter/anaconda3/envs/swift/lib/libuuid.so.1: no version information available (required by /home/jupyter/swift/usr/lib/swift/linux/libFoundation.so)
/home/jupyter/swift/usr/bin/swiftc: /home/jupyter/anaconda3/envs/swift/lib/libuuid.so.1: no version information available (required by /home/jupyter/swift/usr/bin/swiftc)
Compile Swift Module 'FastaiNotebook_10_mixup_ls' (15 sources)
/home/jupyter/swift/usr/bin/swiftc: /home/jupyter/anaconda3/envs/swift/lib/libuuid.so.1: no version information available (required by /home/jupyter/swift/usr/bin/swiftc)

/home/jupyter/swift/usr/bin/swift: /home/jupyter/anaconda3/envs/swift/lib/libuuid.so.1: no version information available (required by /home/jupyter/swift/usr/bin/swift)
/home/jupyter/git/fastai_

/home/jupyter/git/fastai_docs/dev_swift/FastaiNotebook_10_mixup_ls/Sources/FastaiNotebook_10_mixup_ls/01a_fastai_layers.swift:27:65: note: overloads for 'standardDeviation' exist with these partially matching parameter lists: (alongAxes: Int32...), (alongAxes: [Int32])
    func std(squeezingAxes a: [Int]) -> Tensor<Scalar> { return standardDeviation(squeezingAxes: a) }
                                                                ^
/home/jupyter/git/fastai_docs/dev_swift/FastaiNotebook_10_mixup_ls/Sources/FastaiNotebook_10_mixup_ls/01a_fastai_layers.swift:28:73: error: argument labels '(squeezingAxes:)' do not match any available overloads
    func std(squeezingAxes a: Tensor<Int32>) -> Tensor<Scalar> { return standardDeviation(squeezingA
xes: a) }
                                                                        ^                ~~~~~~~~~~~~~~~~~~
/home/jupyter/git/fastai_docs/dev_swift/FastaiNotebook_10_mixup_ls/Sources/FastaiNotebook_10_mixup_ls/01a_fastai_layers.swift:28:73

        if !shuffle { return innerDs.batched(bs)}
                                             ^~
                                             Int64( )
/home/jupyter/git/fastai_docs/dev_swift/FastaiNotebook_10_mixup_ls/Sources/FastaiNotebook_10_mixup_ls/04_callbacks.swift:38:46: error: cannot convert value of type 'Int' to expected argument type 'Int64'
        return innerDs.shuffled(sampleCount: dsCount, randomSeed: seed).batched(bs)
                                             ^~~~~~~
                                             Int64( )
/home/jupyter/git/fastai_docs/dev_swift/FastaiNotebook_10_mixup_ls/Sources/FastaiNotebook_10_mixup_ls/04_callbacks.swift:61:44: error: cannot subscript a value of type 'TensorShape' with an in
dex of type 'Int'
                     trainLen: xTrain.shape[0],
                                           ^
/home/jupyter/git/fastai_docs/dev_swift/FastaiNotebook_10_mixup_ls/Sources/FastaiNotebook_10_mixup_ls/04_callbacks.swift:61:44: note: overloads for '

In [ ]:
import Foundation
import TensorFlow
import FastaiNotebook_10_mixup_ls
// import WAVCat
// import SwiftFFmpeg

In [ ]:
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

In [ ]:
public extension StringTensor {
    // Decode a StringTensor holding a path to a WAV file into (audio: Tensor<Float>, sampleRate: Tensor<Int32>)
    func decodeWav() -> (audio: Tensor<Float>, sampleRate: Tensor<Int32>) {
        return Raw.decodeWav(contents: self)
    }
}

In [ ]:
let wav_file = "m0005_us_m0005_00445.wav"

In [ ]:
let (sig, sr) = StringTensor(readFile: wav_file).decodeWav()

In [ ]:
let audiodata = sig.makeNumpyArray()

In [ ]:
plt.plot(audiodata) 
// plt.axis("off")
plt.show()

In [ ]:
let spec = Raw.audioSpectrogram(sig, 
                                windowSize: 1024, stride: 256, magnitudeSquared: false)

In [ ]:
func showSpec(s: Tensor<Float>) {
    plt.imshow(s.makeNumpyArray()[0], cmap: "plasma")
    plt.show()
    print("Shape: \(s.shape)\nMin:\(s.min()), max: \(s.max()), mean: \(s.mean()), var: \(s.variance())")
}

In [ ]:
showSpec(s: spec)

[TF defaults](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/ops/audio_ops.cc#L147) for MFCCs:
```
    .Attr("upper_frequency_limit: float = 4000")
    .Attr("lower_frequency_limit: float = 20")
    .Attr("filterbank_channel_count: int = 40")
    .Attr("dct_coefficient_count: int = 13")
```

In [ ]:
let mfccs = Raw.mfcc(spectrogram: spec, sampleRate: sr, 
                       upperFrequencyLimit: 8000, lowerFrequencyLimit: 20, 
                       filterbankChannelCount: 40, dctCoefficientCount: 13)

In [ ]:
showSpec(s: mfccs)